In [4]:
from operator import itemgetter

from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.runnables.openai_functions import OpenAIFunctionsRouter

from permchain.connection_inmemory import InMemoryPubSubConnection
from permchain.pubsub import PubSub
from permchain.topic import Topic

In [11]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert coder who only writes python code. For a given user question, you return a python script - and only a python script attempting to implement their given ask. This script should be enclosed in ```python\n...\n``` and you should not have any text before or after that script snippet."),
    ("human", "{task}")
])


In [12]:
chain = prompt | ChatOpenAI(temperature=0) | StrOutputParser()

In [27]:
task = "write a script to scrape the top 10 titles of the top hacker news posts"

In [28]:
result = chain.invoke({"task": task})

In [52]:
functions = [
    {
      "name": "finish",
      "description": "Call this if the code is good as is",
      "parameters": {
        "type": "object",
        "properties": {
          "finished": {
            "const": "true",
            "description": "The city and state, e.g. San Francisco, CA"
          },
        },
        "required": ["finished"]
      }
    },
    {
      "name": "edit",
      "description": "Call this if the output does not look as expected",
      "parameters": {
        "type": "object",
        "properties": {
          "comment": {
            "type": "string",
            "description": "A comment on why the output does not look as expected and suggestion on how to fix it"
          },
        },
        "required": ["comment"]
      }
    },
    {
      "name": "help",
      "description": "Call this if the output does not look as expected and you're not sure what to do next",
      "parameters": {
        "type": "object",
        "properties": {
          "question": {
            "type": "string",
            "description": "A question that the user will answer to help you debug"
          },
        },
        "required": ["question"]
      }
    }
  ]

In [53]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are reviewing some written code a human as written. After they wrote the code, they then ran it. Based on the output of the program, you should either decide to finish (if the code is good) or write a revision."),
    ("human", "This is my task: {task}"),
    ("human", "This is my code:\n\n{code}"),
    ("human", "This is my output:\n\n<output>{output}</output>\n\nDoes this output look as you would expect? If so we can finish. Otherwise, write a new code snippet that produces better results. If you do not finish, you MUST write a replacement code snippet.")
])

In [54]:
critique_chain = prompt | ChatOpenAI(temperature=0, model="gpt-4").bind(functions=functions)

In [55]:
result1 = critique_chain.invoke({
    "task": task,
    "code": result,
    "output": "",
})

In [56]:
result1

AIMessage(content='', additional_kwargs={'function_call': {'name': 'edit', 'arguments': '{\n"comment": "The output is empty. It seems like the code didn\'t run correctly. The code itself looks fine, it might be an issue with the website or the connection. I suggest to add some error handling to the code to catch potential issues."\n}'}}, example=False)

In [57]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are revising some written code a human has written. After they wrote the code, they then ran it. Based on the output of the program, they then suggested some edits."),
    ("human", "This is my task: {task}"),
    ("human", "This is my code:\n\n{code}"),
    ("human", "This is my output:\n\n<output>{output}</output>"),
    ("human", "These are my suggested edits: {edit}"),
    ("human", "Please write a new code snippet taking into account the suggested edits"),
])

In [58]:
revise_chain = prompt | ChatOpenAI(temperature=0, model="gpt-4").bind(functions=functions)

In [59]:
result2 = revise_chain.invoke({
    "task": task,
    "code": result,
    "output": "",
    "edit": result1.additional_kwargs['function_call']['arguments']
})

In [63]:
print(result2.content)

Here is the revised code with added error handling:

```python
import requests
from bs4 import BeautifulSoup

try:
    # Send a GET request to the Hacker News website
    response = requests.get("https://news.ycombinator.com/")

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to get page with status code: {response.status_code}")
        exit()

    # Parse the HTML content of the response
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all the post titles on the page
    post_titles = soup.find_all("a", class_="storylink")

    # Check if any titles were found
    if not post_titles:
        print("No post titles found")
        exit()

    # Print the top 10 titles
    for i, title in enumerate(post_titles[:10]):
        print(f"{i+1}. {title.text}")

except requests.exceptions.RequestException as e:
    # If there was a network problem (e.g. DNS resolution, refused connection, etc), print the error
    print(

In [64]:
import requests
from bs4 import BeautifulSoup

try:
    # Send a GET request to the Hacker News website
    response = requests.get("https://news.ycombinator.com/")

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to get page with status code: {response.status_code}")
        exit()

    # Parse the HTML content of the response
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all the post titles on the page
    post_titles = soup.find_all("a", class_="storylink")

    # Check if any titles were found
    if not post_titles:
        print("No post titles found")
        exit()

    # Print the top 10 titles
    for i, title in enumerate(post_titles[:10]):
        print(f"{i+1}. {title.text}")

except requests.exceptions.RequestException as e:
    # If there was a network problem (e.g. DNS resolution, refused connection, etc), print the error
    print(f"An error occurred: {e}")

No post titles found


In [2]:
import requests
from bs4 import BeautifulSoup

try:
    # Send a GET request to the Hacker News website
    response = requests.get("https://news.ycombinator.com/")

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to get page with status code: {response.status_code}")
        exit()

    # Parse the HTML content of the response
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all the post titles on the page
    post_titles = soup.select(".athing .titleline")

    # Check if any titles were found
    if not post_titles:
        print("No post titles found")
        exit()

    # Print the top 4 titles
    for i, title in enumerate(post_titles[:4]):
        print(f"{i+1}. {title.text}")

except requests.exceptions.RequestException as e:
    # If there was a network problem (e.g. DNS resolution, refused connection, etc), print the error
    print(f"An error occurred: {e}")

1. Strangest Research Vessel Heads for Scrapyard After 60 Years (maritime-executive.com)
2. CNET is deleting old articles to try to improve its Google Search ranking (theverge.com)
3. Temptations of an open-source Chrome extension developer (github.com/extesy)
4. Justapedia, a Wikipedia alternative and aspiring replacement, has gone public (justapedia.org)


In [18]:
post_titles

[]

In [20]:
import requests
from bs4 import BeautifulSoup

def get_top_hn_titles():
    url = "https://news.ycombinator.com/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # The titles are within 'a' tags with class 'storylink'
    titles = soup.find_all('a', class_='storylink')
    
    top_titles = [title.text for title in titles[:10]]
    return top_titles

# Print top 10 titles
for idx, title in enumerate(get_top_hn_titles(), 1):
    print(f"{idx}. {title}")

